# Utiliza o CDB com ML para predizer a localização (lat,lon) das EM:

In [4]:
import pandas as pd
import numpy as np
import math
import os
from tqdm import tqdm

%run UtilsLightGBMFingerPrinting.ipynb

# Geração dos Arquivos Outdoor, Indoor, Indoor-Outdoor:

# Arquivo Outdoor:

In [5]:
X_test = pd.read_csv("databasesTCC/Outdoor_X_test_20%.csv")
y_test = pd.read_csv("databasesTCC/Outdoor_y_test_20%.csv")
y_test['delay_1'] = X_test['delay_1']
y_test['delay_2'] = X_test['delay_2']
y_test['delay_3'] = X_test['delay_3']
y_test['delay_12'] = X_test['delay_12']
y_test['delay_13'] = X_test['delay_13']
y_test['delay_23'] = X_test['delay_23']
y_test['idx'] = X_test['idx']
y_test = y_test.head(306)
y_test.to_csv("./Test/Outdoor_Test.csv", index=False)
y_test.head()

,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,delay_1,delay_2,delay_3,delay_12,delay_13,delay_23,idx
0,-94.588571,-97.102857,-94.058571,-96.128333,-92.27500,-90.411429,-77.951667,-73.190,-94.7525,6,5,1,1,5,4,7080
1,-103.010000,-84.498571,-97.591429,-95.010000,-98.83750,-97.332857,-100.650000,-101.620,-94.4075,3,3,6,0,-3,-3,9438
2,-95.202857,-86.280000,-93.030000,-53.330000,-84.96500,-81.080000,-90.826667,-94.342,-98.9125,4,2,6,2,-2,-4,8840
3,-92.422619,-90.242143,-93.064643,-80.359286,-87.13750,-86.910952,-91.338333,-93.248,-97.2550,5,2,5,3,0,-3,7629
4,-86.227143,-86.231429,-87.885714,-72.830000,-80.56625,-85.737143,-90.191667,-90.704,-97.6000,5,2,5,3,0,-3,7988


# Arquivo Indoor:

In [6]:
X_test = pd.read_csv("databasesTCC/indoor_X_test_20%.csv")
y_test = pd.read_csv("databasesTCC/indoor_y_test_20%.csv")
y_test['delay_1'] = X_test['delay_1']
y_test['delay_2'] = X_test['delay_2']
y_test['delay_3'] = X_test['delay_3']
y_test['delay_12'] = X_test['delay_12']
y_test['delay_13'] = X_test['delay_13']
y_test['delay_23'] = X_test['delay_23']
y_test['idx'] = X_test['idx']
y_test = y_test.head(306)
y_test.to_csv("./Test/Indoor_Test.csv", index=False)
y_test.head()

,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,delay_1,delay_2,delay_3,delay_12,delay_13,delay_23,idx
0,-101.196667,-115.00,-96.04,-115.00,-96.790000,-96.100000,-94.420000,-90.220000,-115.0,6,4,3,2,3,1,4395
1,-100.830000,-115.00,-98.56,-115.00,-94.796667,-97.648571,-99.255000,-96.904000,-115.0,6,3,3,3,3,0,3270
2,-98.748000,-97.85,-98.61,-98.49,-90.710000,-90.820000,-100.030000,-95.910000,-115.0,6,3,3,3,3,0,2026
3,-115.000000,-115.00,-115.00,-115.00,-115.000000,-115.000000,-91.107143,-93.163333,-115.0,6,4,3,2,3,1,420
4,-98.748000,-99.67,-98.61,-97.80,-88.115000,-90.684286,-100.030000,-91.585000,-115.0,6,3,3,3,3,0,1984


# Arquivo Indoor-Outdoor:

In [7]:
X_test_outdoor = pd.read_csv("databasesTCC/Outdoor_X_test_20%.csv")
y_test_outdoor = pd.read_csv("databasesTCC/Outdoor_y_test_20%.csv")
y_test_outdoor['delay_1'] = X_test_outdoor['delay_1']
y_test_outdoor['delay_2'] = X_test_outdoor['delay_2']
y_test_outdoor['delay_3'] = X_test_outdoor['delay_3']
y_test_outdoor['delay_12'] = X_test_outdoor['delay_12']
y_test_outdoor['delay_13'] = X_test_outdoor['delay_13']
y_test_outdoor['delay_23'] = X_test_outdoor['delay_23']
y_test_outdoor['idx'] = X_test_outdoor['idx']
y_test_outdoor = y_test_outdoor.tail(153)

X_test_indoor = pd.read_csv("databasesTCC/indoor_X_test_20%.csv")
y_test_indoor = pd.read_csv("databasesTCC/indoor_y_test_20%.csv")
y_test_indoor['delay_1'] = X_test_indoor['delay_1']
y_test_indoor['delay_2'] = X_test_indoor['delay_2']
y_test_indoor['delay_3'] = X_test_indoor['delay_3']
y_test_indoor['delay_12'] = X_test_indoor['delay_12']
y_test_indoor['delay_13'] = X_test_indoor['delay_13']
y_test_indoor['delay_23'] = X_test_indoor['delay_23']
y_test_indoor['idx'] = X_test_indoor['idx']
y_test_indoor = y_test_indoor.tail(153)


df_indoor_outdoor = pd.concat([y_test_outdoor, y_test_indoor])
df_indoor_outdoor.to_csv("./Test/Indoor-Outdoor_Test.csv", index=False)
df_indoor_outdoor.head()

,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,delay_1,delay_2,delay_3,delay_12,delay_13,delay_23,idx
460,-92.876190,-98.737143,-88.461429,-101.7375,-101.5175,-95.780000,-81.120000,-83.474,-93.0900,6,5,1,1,5,4,7493
461,-95.090000,-88.724286,-93.744286,-67.1400,-82.1950,-85.748571,-89.860000,-96.776,-98.9125,4,1,6,3,-2,-5,8688
462,-95.141429,-97.387143,-93.847143,-98.1650,-88.2125,-92.814286,-75.156667,-56.370,-90.6500,6,5,1,1,5,4,7158
463,-90.195714,-99.284286,-91.990000,-101.0550,-92.8475,-94.597143,-88.450000,-74.904,-74.1000,7,6,1,1,6,5,6860
464,-83.602857,-73.790000,-85.271786,-54.9200,-67.5475,-58.860000,-90.791667,-88.616,-97.2550,6,1,5,5,1,-4,7453


In [8]:
X_test = pd.read_csv("./CDB/CDB_20_LightGBM_Complete.csv")
X_test.head()

,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,delay_1,delay_2,delay_3,delay_12,delay_13,delay_23,lat,lon
0,-95.257429,-101.256458,-90.994183,-82.774777,-88.158168,-98.760354,-93.745712,-86.044939,-70.074586,8,8,2,0,6,6,-8.05955,-34.959850
1,-95.257429,-101.256458,-90.994291,-82.774777,-88.158559,-98.760354,-93.745712,-81.075098,-70.072108,8,8,2,0,6,6,-8.05955,-34.959648
2,-95.257429,-101.256444,-90.994291,-82.774777,-88.158104,-97.724096,-93.748641,-81.075098,-70.072108,8,8,2,0,6,6,-8.05955,-34.959446
3,-95.257429,-101.256466,-90.994291,-82.774802,-88.124843,-97.724096,-93.748641,-65.473641,-70.072108,8,8,2,0,6,6,-8.05955,-34.959244
4,-94.395072,-101.260548,-91.171270,-82.774799,-88.055176,-97.708748,-93.749146,-65.473651,-87.845521,8,7,2,1,6,5,-8.05955,-34.959042


In [9]:
FILES = [
    # Arquivos de Teste Outdoor Só possui RSSIs, delays, idx
    #(7,'./Test/Outdoor_Test.csv'), # Usar CDB_20_LightGBM_Complete
    #(8,'./Test/Indoor_Test.csv'), # Usar CDB_20_LightGBM_Complete
    #(9,'./Test/Indoor-Outdoor_Test.csv'), # Usar CDB_20_LightGBM_Complete
    #(10,'./Test/Outdoor_Test.csv'), # Usar CDB_20_SVR_Complete
    (11,'./Test/Indoor_Test.csv'), # Usar CDB_20_SVR_Complete
    (12,'./Test/Indoor-Outdoor_Test.csv') # Usar CDB_20_SVR_Complete
]
#TEST_DATABASE = "./CDB/CDB_20_LightGBM_Complete.csv"
TEST_DATABASE = "./CDB/CDB_20_SVR_Complete.csv"

In [10]:
def get_nearest_point(test_database, point):
    """
    Encontra o ponto no fingerprint que possui a menor distância euclidiana para o ponto passado como argumento
    """
    min_dist = None
    lat = None
    lon = None
    
    for idx, point_fg in test_database.iterrows():
        fg = point_fg[:-2]
        dist = euclidean_distance(point[:-1], fg)
        
        if (min_dist == None) or (min_dist > dist):
            min_dist = dist
            lat = point_fg.lat
            lon = point_fg.lon

    return lat, lon

def calculate_coords_th(fingerprint, test_database):
    """
    Calcula as coordenadas usando um fingerprint e retorna um dataframe contendo
    latitude, longitude e pontoId
    """
    points_dict = {'lat_pred': [], 'lon_pred': [], 'idx': []}
    
    """"
    for idx, point in tqdm(test_database.iterrows()):
        lat, lon = get_nearest_point(fingerprint, point)
        points_dict['lat_pred'].append(lat)
        points_dict['lon_pred'].append(lon)
        points_dict['idx'].append(point.idx)
    """
    for idx, point in tqdm(fingerprint.iterrows()):
        lat, lon = get_nearest_point(test_database, point)
        points_dict['lat_pred'].append(lat)
        points_dict['lon_pred'].append(lon)
        points_dict['idx'].append(point.idx)
    
    
    return pd.DataFrame(points_dict)
    
def generate_theorical_results():
    """
    Calcula as latitudes e longitudes estimadas dos pontos da base de teste usando cada um dos fingerprints
    com as dimensões listadas no vetor DIM
    """
    test_database = pd.read_csv(TEST_DATABASE)
    for method, file in FILES:
        fingerprint = pd.read_csv(file)
        points_df = calculate_coords_th(fingerprint, test_database)
        points_df.to_csv("./FinalResult/Resultados_Metodo_{}.csv".format(method), index=False)

In [11]:
# gera os resultados teoricos
generate_theorical_results()

306it [23:08,  3.73s/it]
306it [19:02,  3.73s/it]


In [45]:
casa = ["rssi_1_1","rssi_1_2","rssi_1_3","rssi_2_1","rssi_2_2","rssi_2_3","rssi_3_1","rssi_3_2","rssi_3_3","delay_1","delay_2","delay_3","delay_12","delay_13","delay_23","idx"]
casa[:-7]

['rssi_1_1',
 'rssi_1_2',
 'rssi_1_3',
 'rssi_2_1',
 'rssi_2_2',
 'rssi_2_3',
 'rssi_3_1',
 'rssi_3_2',
 'rssi_3_3']

In [47]:
casa = ["rssi_1_1","rssi_1_2","rssi_1_3","rssi_2_1","rssi_2_2","rssi_2_3","rssi_3_1","rssi_3_2","rssi_3_3","delay_1","delay_2","delay_3","delay_12","delay_13","delay_23","lat","lon"]
casa[:-8]

['rssi_1_1',
 'rssi_1_2',
 'rssi_1_3',
 'rssi_2_1',
 'rssi_2_2',
 'rssi_2_3',
 'rssi_3_1',
 'rssi_3_2',
 'rssi_3_3']